In [ ]:
!pip install selenium --upgrade

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests
import time

def scraper (email, password, keyword, num_pages):
    # Initialize the browser
    #PROXY = "37.207.45.15:48678"  # For example, "192.168.88.88:8080"

    chrome_options = webdriver.ChromeOptions()
    #chrome_options.add_argument('--proxy-server=%s' % PROXY)

    browser = webdriver.Chrome(options=chrome_options, )  # Ensure chromedriver is in PATH or specify its path here.

    # Navigate to Google Scholar
    browser.get("https://www.corriere.it/economia/")

    time.sleep(5)  # This may need adjusting.

    # Find the search input and input your keyword
    #user = browser.find_element(By.NAME, "email")
    #user.send_keys(email)
    #psw = browser.find_element(By.NAME, "password")
    #psw.send_keys(password)
    #psw.send_keys(Keys.RETURN)

    # Wait for results to load
    
    #browser.get("https://z-lib.is/")
    #search = browser.find_element(By.NAME, "q")
    #search.send_keys(keyword)
    #search.send_keys(Keys.RETURN)

    #pages_scraped = 0
    
    while True:
        # Parse the loaded page
        soup = BeautifulSoup(browser.page_source, "html.parser")
        

        
        
        # Extract PDF links
        pdf_links = [link['href'] for link in soup.find_all('a', href=True) if link['href'].endswith('.pdf')]
        print(pdf_links)
        
        # Download each PDF
        for pdf_link in pdf_links:
            
            time.sleep(3)
            response = browser.get(pdf_link, stream=True)
            with open('Users/joanvelja/Documents/1000Kelvin/data/papers/scholar/' + pdf_link.split('/')[-1], 'wb') as pdf:
                for chunk in response.iter_content(chunk_size=8192):
                    pdf.write(chunk)

        pages_scraped += 1
        if num_pages and pages_scraped >= num_pages:
            break
        
        # Move to the next page
        try:
            next_button = browser.find_element(By.LINK_TEXT, "Next")
            next_button.click()
            time.sleep(5)
        except:
            # No more pages left
            break

    browser.quit()


In [ ]:
class add_argument:
    """
    Disable Blink features and automation indicator to imitate human interaction with navigator. Return Options.
    """
    def add_argument_options():
        """
        add_argument_options : None -> Options()
        add_argument_options() Disable Blink features and automation indicator to imitate human interaction with navigator. Return Options.
        """
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--kiosk")
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-blink-features")
        chrome_options.add_argument('--profile-directory=Default')
        chrome_options.add_argument("--disable-plugins-discovery")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        chrome_options.add_experimental_option("detach", True)

        return chrome_options

In [ ]:
!pip install cloudscraper

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

def download_pdf_from_link(browser, link_url):
    """Navigates to the provided link and downloads the PDF found on the page."""
    browser.get(link_url)

    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    time.sleep(5)
    # First, find the div with no class
    option_2_div = browser.find_element(By.XPATH, '//div[text()="🐢 Slow & external downloads"]/following-sibling::ul/li[2]/a')
    link_option_2 = option_2_div.get_attribute('href')
    print(link_option_2)
    browser.get(link_option_2)

    # Find the PDF link on the target page. Adjust the selector accordingly.
    pdf_link_element = browser.find_element(By.CSS_SELECTOR, 'a[href$=".pdf"]')
    pdf_link = pdf_link_element.get_attribute('href')

    response = requests.get(pdf_link, stream=True)
    with open('/Users/joanvelja/Documents/1000Kelvin/data/papers/anna/' + pdf_link.split('/')[-1], 'wb') as pdf:
        for chunk in response.iter_content(chunk_size=8192):
            pdf.write(chunk)

def scraper_v2(base_url):
    chrome_options = add_argument.add_argument_options()
    browser = webdriver.Chrome(options=chrome_options)

    browser.get(base_url)
    time.sleep(3)

    # Parse the loaded page
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract links from the target divs
    wrapper_div = soup.find('div', class_='min-w-[0]')
    if wrapper_div:
        mb4_divs = wrapper_div.find_all('div', class_='mb-4')
        
        for div in mb4_divs:
            for link in div.find_all('a', href=True):
                print(link['href'])
                goto = "https://annas-archive.org/" + link['href']
                download_pdf_from_link(browser, goto)

    browser.quit()

In [ ]:
import requests
import cloudscraper
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def download_pdf_from_link(browser, link_url):
    """Navigates to the provided link and downloads the PDF found on the page."""
    browser.get(link_url)
    time.sleep(5)

    # First, find the div with no class
    option_2_div = browser.find_element(By.XPATH, '//div[text()="🐢 Slow & external downloads"]/following-sibling::ul/li[2]/a')
    link_option_2 = option_2_div.get_attribute('href')
    print(link_option_2)

    # Here we use cloudscraper to bypass the Cloudflare challenge.
    scraper = cloudscraper.create_scraper(browser='chrome')
    response = scraper.get(link_option_2)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the PDF link from the soup content. Adjust the selector accordingly.
    pdf_link_element = soup.select_one('a[href$=".pdf"]')
    pdf_link = pdf_link_element['href']

    response = requests.get(pdf_link, stream=True)
    with open('/Users/joanvelja/Documents/1000Kelvin/data/papers/anna/' + pdf_link.split('/')[-1], 'wb') as pdf:
        for chunk in response.iter_content(chunk_size=8192):
            pdf.write(chunk)

def scraper_v2(base_url):
    # Assuming `add_argument` is a module or function you've created or imported elsewhere
    chrome_options = add_argument.add_argument_options()
    browser = webdriver.Chrome(options=chrome_options)

    browser.get(base_url)
    time.sleep(3)

    # Parse the loaded page
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract links from the target divs
    wrapper_div = soup.find('div', class_='min-w-[0]')
    if wrapper_div:
        mb4_divs = wrapper_div.find_all('div', class_='mb-4')
        
        for div in mb4_divs:
            for link in div.find_all('a', href=True):
                print(link['href'])
                goto = "https://annas-archive.org/" + link['href']
                download_pdf_from_link(browser, goto)

    browser.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from bs4 import BeautifulSoup
import requests
import time
import os
from concurrent.futures import ThreadPoolExecutor

def download_pdf(pdf_link, file_path):
    try:
        response = requests.get(pdf_link, stream=True)
        response.raise_for_status()  # Check if the request was successful
    except requests.RequestException as e:
        print(f"Failed to download {pdf_link}. Reason: {e}")
        return

    try:
        with open(file_path, 'wb') as pdf:
            for chunk in response.iter_content(chunk_size=8192):
                pdf.write(chunk)
    except IOError as e:
        print(f"Failed to save PDF. Reason: {e}")

def scraper(keyword, num_pages):
    try:
        # Initialize the browser
        # PROXY = "37.207.45.15:48678"
        chrome_options = add_argument.add_argument_options()
        # chrome_options.add_argument('--proxy-server=%s' % PROXY)
        browser = webdriver.Chrome()

        # Navigate to Arxiv
        browser.get("https://arxiv.org/")

        # Locate the search input
        try:
            search_box = browser.find_element(By.NAME, "query")
        except NoSuchElementException:
            print("Failed to find the search box element.")
            browser.quit()
            return
        
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.RETURN)

        # Wait for results to load
        time.sleep(1.5)

        pages_scraped = 0

        while True:
            # Parse the loaded page
            soup = BeautifulSoup(browser.page_source, "html.parser")

            # Extract PDF links
            pdf_links = [link['href'] for link in soup.find_all('a', href=True) if 'pdf' in link['href']]
            print(pdf_links)

            # Download each PDF in parallel
            with ThreadPoolExecutor(max_workers=5) as executor:
                for pdf_link in pdf_links:
                    pdf_link += '.pdf'
                    file_name = pdf_link.split('/')[-1]
                    file_path = '/Users/joanvelja/Documents/1000Kelvin/data/papers/arxiv/processed/' + file_name
                    if not os.path.exists(file_path):
                        executor.submit(download_pdf, pdf_link, file_path)
                    else:
                        print(f"Skipping {file_name} as it already exists.")

            pages_scraped += 1
            if num_pages and pages_scraped >= num_pages:
                break

            # Move to the next page
            try:
                next_button = browser.find_element(By.LINK_TEXT, "Next")
                next_button.click()
                time.sleep(5)
            except NoSuchElementException:
                # No more pages left
                break

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")

scraper('Additive Manufacturing', 25)


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--proxy-server=%s' % PROXY)

browser = webdriver.Chrome(options=chrome_options, )  # Ensure chromedriver is in PATH or specify its path here.

# Navigate to Google Scholar
browser.get("https://www.corriere.it/economia/")

time.sleep(3)  # This may need adjusting.

# Find the search input and input your keyword
browser.find_element(By.ID, "privacy-cp-wall-accept").click()

time.sleep(3)

WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'a')))

# Then extract the links
link_elements = browser.find_elements(By.TAG_NAME, 'a')
links = [element.get_attribute('href') for element in link_elements if element.get_attribute('href')]


for l in links:
    print(l)

In [7]:
def surf_and_cookies(url, browser):
    browser.get(url)
    time.sleep(3)
    try:
        # If the cookie banner is present, click the accept button
        browser.find_element(By.ID, "privacy-cp-wall-accept").click()
    except:
        # Otherwise, just skip it
        pass
    time.sleep(3)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'a')))
    link_elements = browser.find_elements(By.TAG_NAME, 'a')
    links = [element.get_attribute('href') for element in link_elements if element.get_attribute('href') and
             element.get_attribute('href').startswith(url)]
    return links

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Setup WebDriver
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--proxy-server=%s' % PROXY)
# incognito window, no extensions, lowest bot detection policies implemented

# Embed extensions already in my chrome instance
#chrome_options.add_argument('--user-data-dir=/Users/joanvelja/Library/Application Support/Google/Chrome/Default')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(options=chrome_options, )  # Ensure chromedriver is in PATH or specify its path here.
driver.get('https://www.corriere.it/')

time.sleep(5)  # This may need adjusting.

# Find the search input and input your keyword
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'privacy-cp-wall-accept')))
driver.find_element(By.ID, "privacy-cp-wall-accept").click()


time.sleep(5)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'a')))

# Find all elements with <a> tag
link_elements = driver.find_elements(By.TAG_NAME, 'a')

# Filter links
desired_links = [element.get_attribute('href') for element in link_elements if element.get_attribute('href') and 
                 (element.get_attribute('href').startswith("https://www.corriere.it/economia/") or 
                  element.get_attribute('href').startswith("https://www.corriere.it/politica/"))]

# Remove duplicates
desired_links = list(set(desired_links))

econolinks = surf_and_cookies('https://www.corriere.it/economia/', driver)
politilinks = surf_and_cookies('https://www.corriere.it/politica/', driver)


# Merge lists and remove duplicates
desired_links = list(set(econolinks + politilinks))

while len(desired_links) < 300:
    for link in desired_links:
        print(f"Currently on {link}")
        # Get links from page
        s = surf_and_cookies(link, driver)
        # Add to desired links
        desired_links.extend(s)
        # Remove duplicates
        desired_links = list(set(desired_links))
        print(f"Found {len(desired_links)} links.")


Currently on https://www.corriere.it/politica/#
Found 140 links.
Currently on https://www.corriere.it/economia/moda-business/23_dicembre_22/confindustria-moda-il-2023-32percento-fatturato-ma-margini-calo-d2645cc0-a0c0-11ee-8a50-aa124a9df6fa.shtml
Found 142 links.
Currently on https://www.corriere.it/economia/innovazione/intelligenza-artificiale/
Found 155 links.
Currently on https://www.corriere.it/economia/moda-business/23_dicembre_21/prada-compra-l-edificio-suo-negozio-new-york-fifth-avenue-425-milioni-dollari-0286e484-9ff6-11ee-b1cc-35e5cecb801b.shtml
Found 157 links.
Currently on https://www.corriere.it/economia/finanza/23_dicembre_21/banche-piu-solide-europa-secondo-bce-credem-primo-posto-lista-completo-c42407c2-9f68-11ee-9c51-1d1800643026.shtml
Found 159 links.
Currently on https://www.corriere.it/politica/23_dicembre_20/mattarella-la-russa-premierato-22db094e-9ea8-11ee-bb1f-834db2b90026.shtml
Found 160 links.
Currently on https://www.corriere.it/economia/aziende/23_dicembre_22/a

KeyboardInterrupt: 

In [17]:
# Only keep links that end with html
desired_links = [link for link in desired_links if link.endswith('html')]

In [18]:
desired_links

['https://www.corriere.it/economia/tasse/cards/stipendi-cosi-aumenti-salariali-possono-azzerare-benefici-taglio-cuneo-conti/paradosso-modalita-fasce_principale.shtml',
 'https://www.corriere.it/economia/consumi/23_dicembre_19/ferragni-uova-pasqua-come-pandoro-piu-un-milione-dolci-preziosi-che-ha-donato-36-mila-euro-fc0ff5f8-9e44-11ee-bb1f-834db2b90026.shtml',
 'https://www.corriere.it/economia/opinioni/23_novembre_28/magneti-marelli-l-occasione-persa-perche-cessione-all-estero-andava-evitata-37f16afe-8d18-11ee-b4a7-347e05f6b33b.shtml',
 'https://www.corriere.it/economia/innovazione/intelligenza-artificiale/23_dicembre_06/accenture-milano-roma-due-centri-l-ai-generativa-rivolti-aziende-cosa-faranno-73d04eca-945f-11ee-bf17-27011c9bfd8d.shtml',
 'https://www.corriere.it/economia/professionisti/consulenti-del-lavoro/cards/cashback-fiscale-cos-spese-rimborsate-subito-conto-corrente-senza-aspettare-730/che-cosa-c-nuovo_principale.shtml',
 'https://www.corriere.it/economia/finanza/23_dicembre

In [ ]:
# crawl all links within a url and save them to a list
def crawl_links(url, browser):
    browser.get(url)
    time.sleep(3)
    try:
        # If the cookie banner is present, click the accept button
        browser.find_element(By.ID, "privacy-cp-wall-accept").click()
    except:
        # Otherwise, just skip it
        pass
    time.sleep(3)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'a')))
    link_elements = browser.find_elements(By.TAG_NAME, 'a')
    links = [element.get_attribute('href') for element in link_elements if element.get_attribute('href') and
             element.get_attribute('href').startswith(url)]
    return links



In [ ]:
# crawl all links within a url and all the sublinks within those links, save them to a list and remove duplicates

def crawl_links(url, browser):
    browser.get(url)
    time.sleep(3)
    try:
        # If the cookie banner is present, click the accept button
        browser.find_element(By.ID, "privacy-cp-wall-accept").click()
    except:
        # Otherwise, just skip it
        pass
    time.sleep(3)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'a')))
    link_elements = browser.find_elements(By.TAG_NAME, 'a')
    links = [element.get_attribute('href') for element in link_elements if element.get_attribute('href') and
             element.get_attribute('href').startswith(url)]
    return links
